In [1]:
import pypsa
import numpy as np
import pandas as pd
import os
import matplotlib
matplotlib.use("TkAgg")
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
plt.rcParams['figure.dpi']= 400
plt.style.use('bmh')
%matplotlib inline
pd.set_option('max_columns', None)
pd.get_option("max_columns")
#pd.reset_option(“max_columns”)

In [2]:
path_zone='D:\\Python\\PyPSA\\Luca\\zonal_nodal_networks\\2018\\zonal_1024_costs2018.nc'

In [3]:
n = pypsa.Network(path_zone)

INFO:pypsa.io:Imported network zonal_1024_costs2018.nc has buses, carriers, generators, links, loads, storage_units


In [4]:
# start='2018-01-01 00:00:00'
# end='2018-01-07 23:00:00'
# timespan = pd.date_range(start=start, end=end, freq='H') 

In [5]:
# test test end

In [6]:
n52 = pypsa.Network()

In [7]:
# define snapshot for n52
n52.set_snapshots(range(1,53))
n52.snapshots

RangeIndex(start=1, stop=53, step=1, name='snapshot')

In [8]:
# assign bus network to new 52 network (without changes)
n52.buses=n.buses

In [9]:
# assign loads to new network

In [10]:
n52.madd('Load',n.loads.bus.unique(),
        bus=n.loads.bus.unique(),
        p_set=0,
        q_set=0,
        sign=-1)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Index(['AT', 'BE', 'BG', 'CH', 'ITn', 'CZ', 'DE', 'DKw', 'DKe', 'EE', 'ES',
       'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'ITsic', 'ITs', 'ITcs',
       'ITcn', 'ITsar', 'LT', 'LU', 'LV', 'MK', 'NL', 'NOm', 'NOs', 'NOn',
       'PL', 'PT', 'RO', 'SE1', 'SE3', 'SE2', 'SE4', 'SI', 'SK'],
      dtype='object')

Aggregate loads and sum over week

In [11]:
# first get dictionary {'bus': [list of loads belonging to respective bus]}
load_dict = dict()
for bus in n.loads.bus.unique():
    load_list = []
    for load in n.loads.index:        
        if n.loads.bus[load]==bus:
            load_list.append(load)
    load_dict[bus]=load_list

In [12]:
hours = [] # length of week in hours; turns out last 'week' is just one day so should be if 53 should be 52
for w in range(54): 
    if w !=0:
        week = []
        for time in n.snapshots.strftime('%U'): # this ???!?!??!?
            if int(time)==w:
                week.append(w)
        hours.append(len(week))


In [13]:
# generate list with 168* 1, 168*2
weeks = [k for j in [[i]*168 for i in range(1,53)] for k in j ]
if len(weeks)<len(n.snapshots):
    last_hours = len(n.snapshots)-len(weeks)
#    for i in [52]*last_hours:
#        weeks.append(i)
# list of hours in weeks (all 168 except last week which is last_hours longer)
week_length = [168]*52
#week_length[-1]=168+last_hours

In [14]:
len(weeks)
last_hours

24

In [15]:
# # drop last hours from all the time parameters starting here with load
# n.loads_t.p_set.drop(n.loads_t.p_set.tail(last_hours).index,inplace=True)

In [16]:
# # get help column with week indeed from 1 to 52 (including)
# n.loads_t.p_set['week']=weeks
# ### note:
# # # this is to sum by week
# # n.loads_t.p_set.groupby(n.loads_t.p_set.week).sum()
# # # this is to sum over all loads connected to node
# # n.loads_t.p_set[load_dict[bus]].sum(axis=1)
# df = pd.DataFrame()
# for bus in load_dict:
#     df[bus]=(n.loads_t.p_set.groupby(n.loads_t.p_set.week).sum()[load_dict[bus]].sum(axis=1)) # gives sum for all buses
# # set index to be the same as snapshots
# df.index = range(1,53)
# # assign help df to load p_set
# n52.loads_t.p_set=df

In [17]:
# get help column with week indeed from 1 to 52 (including)
#n.loads_t.p_set['week']=weeks
### note:
# # this is to sum by week
# n.loads_t.p_set.groupby(n.loads_t.p_set.week).sum()
# # this is to sum over all loads connected to node
# n.loads_t.p_set[load_dict[bus]].sum(axis=1)
df_help = pd.DataFrame()
df_help = n.loads_t.p_set
df_help.drop(df_help.tail(last_hours).index,inplace=True)
df_help['week']=weeks

df = pd.DataFrame()

for bus in load_dict:
    df[bus]=(df_help.groupby(df_help.week).sum()[load_dict[bus]].sum(axis=1)) # gives sum for all buses
# set index to be the same as snapshots
df.index = range(1,53)
# assign help df to load p_set
n52.loads_t.p_set=df

In [18]:
##df_help.groupby(df_help.week).sum()[load_dict['DE']].sum(axis=1).sum()/1e6 # total annual demand in TWh

In [19]:
# potentially also add property 'bus' to load df?

Generator aggregation

In [20]:
# only my load shedding generators had no carrier assigned so fix this
for gen,carrier in zip(n.generators.index,n.generators.carrier):
    if not carrier:
        n.generators.carrier[gen]='load_shed'

C:\Users\Luca01\AppData\Local\Temp/ipykernel_3808/254973365.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
#n.generators_t.p_max_pu

In [22]:
# sum generator capacities by carrier and bus & get mean marginal cost for respective technology
df_help = pd.DataFrame()
df_help = n.generators.groupby([n.generators.bus,n.generators.carrier],as_index=False)['p_nom'].sum()
#df_help.p_nom=df_help.p_nom*168
df_help['marginal_cost']=n.generators.groupby([n.generators.bus,n.generators.carrier],as_index=False)['marginal_cost'].mean()['marginal_cost']
df_help['efficiency']=n.generators.groupby([n.generators.bus,n.generators.carrier],as_index=False)['efficiency'].mean()['efficiency']
df_help

,bus,carrier,p_nom,marginal_cost,efficiency
0,AT,CCGT,2481.660000,53.79,0.500
1,AT,OCGT,1313.500000,65.98,0.390
2,AT,coal,991.539415,45.91,0.464
3,AT,load_shed,200000.000000,10000.00,1.000
4,AT,onwind,2574.537530,1.50,1.000
...,...,...,...,...,...
364,SK,load_shed,90000.000000,10000.00,1.000
365,SK,nuclear,1940.000000,14.09,0.337
366,SK,onwind,3.140000,1.50,1.000
367,SK,ror,641.330000,3.00,0.900


In [23]:
gen_agg = ['CCGT','OCGT','coal', 'nuclear','biomass','lignite', 'oil','geothermal','load_shed']
gen_keep = ['onwind', 'ror', 'solar','offwind-ac', 'offwind-dc']

In [24]:
# add conventional generators
for gen in df_help.index:
    if df_help.carrier[gen] in gen_agg:
        n52.add('Generator', '{} {}'.format(df_help.bus[gen],df_help.carrier[gen]),
                p_nom = df_help.p_nom[gen],
                marginal_cost = df_help.marginal_cost[gen],
                efficiency = df_help.efficiency[gen],
                bus = df_help.bus[gen],
                carrier = df_help.carrier[gen])

In [25]:
for gen in n.generators.index:
    if n.generators.carrier[gen] in gen_keep:
        n52.add('Generator', '{}'.format(gen),
                p_nom = n.generators.p_nom[gen],
                marginal_cost = n.generators.marginal_cost[gen],
                efficiency = n.generators.efficiency[gen],
                bus = n.generators.bus[gen],
                carrier = n.generators.carrier[gen])        

In [26]:
# # get help column with week indeed from 1 to 52 (including)
# n.loads_t.p_set['week']=weeks
# ### note:
# # # this is to sum by week
# # n.loads_t.p_set.groupby(n.loads_t.p_set.week).sum()
# # # this is to sum over all loads connected to node
# # n.loads_t.p_set[load_dict[bus]].sum(axis=1)
# df = pd.DataFrame()
# for bus in load_dict:
#     df[bus]=(n.loads_t.p_set.groupby(n.loads_t.p_set.week).sum()[load_dict[bus]].sum(axis=1)) # gives sum for all buses
# # set index to be the same as snapshots
# df.index = range(1,53)
# # assign help df to load p_set
# n52.loads_t.p_set=df

In [27]:
len(weeks)

8736

In [28]:
(last_hours)

24

In [29]:
df = pd.DataFrame()
df = n.generators_t.p_max_pu
df.drop(df.tail(last_hours).index,inplace=True)
df
df['weeks']=weeks
df = df.groupby(df.weeks).mean()
df.index = range(1,53)
# assign help df to RES generators pmaxpu
n52.generators_t.p_max_pu=df

In [30]:
n52.generators.p_nom=n52.generators.p_nom*168

In [31]:
# # sum generators by carrier and bus
# for gen,p_nom,carrier in zip(n.generators.index,n.generators.p_nom,n.generators.carrier):
#     if carrier == 'ror':
#         print(gen,p_nom)

Storage Units (Hydro and PHS)

In [32]:
n52.storage_units = n.storage_units
n52.storage_units.p_nom=n52.storage_units.p_nom*168
n52.storage_units.max_hours=n52.storage_units.max_hours/168

In [33]:
df = pd.DataFrame()
df = n.storage_units_t.inflow
df.drop(df.tail(last_hours).index,inplace=True)
df['weeks']=weeks
df = df.groupby(df.weeks).sum()
df.index = range(1,53)
# assign help df to storage inflow
n52.storage_units_t.inflow=df
n52.storage_units_t.inflow

,AT0 0 hydro,AT0 10 hydro,AT0 13 hydro,AT0 14 hydro,AT0 15 hydro,AT0 2 hydro,AT0 3 hydro,AT0 4 hydro,AT0 5 hydro,AT0 6 hydro,AT0 7 hydro,AT0 9 hydro,BE0 6 hydro,BE0 9 hydro,BG0 1 hydro,BG0 2 hydro,BG0 3 hydro,BG0 4 hydro,BG0 7 hydro,BG0 8 hydro,CH0 0 hydro,CH0 1 hydro,CH0 10 hydro,CH0 13 hydro,CH0 14 hydro,CH0 15 hydro,CH0 16 hydro,CH0 17 hydro,CH0 3 hydro,CH0 4 hydro,CH0 5 hydro,CH0 7 hydro,CH0 8 hydro,CH0 9 hydro,CZ0 17 hydro,CZ0 19 hydro,CZ0 2 hydro,CZ0 3 hydro,CZ0 4 hydro,CZ0 7 hydro,CZ0 8 hydro,DE0 28 hydro,DE0 57 hydro,DE0 72 hydro,ES0 1 hydro,ES0 11 hydro,ES0 12 hydro,ES0 16 hydro,ES0 17 hydro,ES0 18 hydro,ES0 2 hydro,ES0 20 hydro,ES0 21 hydro,ES0 24 hydro,ES0 26 hydro,ES0 28 hydro,ES0 33 hydro,ES0 38 hydro,ES0 39 hydro,ES0 4 hydro,ES0 41 hydro,ES0 46 hydro,ES0 47 hydro,ES0 49 hydro,ES0 5 hydro,ES0 52 hydro,ES0 53 hydro,ES0 55 hydro,ES0 56 hydro,ES0 57 hydro,ES0 59 hydro,ES0 6 hydro,ES0 61 hydro,ES0 62 hydro,ES0 64 hydro,ES0 65 hydro,ES0 67 hydro,ES0 68 hydro,ES0 69 hydro,ES0 7 hydro,ES0 70 hydro,ES0 9 hydro,FI3 1 hydro,FI3 10 hydro,FI3 12 hydro,FI3 16 hydro,FI3 19 hydro,FI3 21 hydro,FI3 27 hydro,FI3 4 hydro,FI3 6 hydro,FI3 7 hydro,FR0 0 hydro,FR0 12 hydro,FR0 122 hydro,FR0 126 hydro,FR0 13 hydro,FR0 133 hydro,FR0 16 hydro,FR0 20 hydro,FR0 21 hydro,FR0 25 hydro,FR0 27 hydro,FR0 29 hydro,FR0 3 hydro,FR0 34 hydro,FR0 37 hydro,FR0 44 hydro,FR0 45 hydro,FR0 5 hydro,FR0 51 hydro,FR0 55 hydro,FR0 6 hydro,FR0 62 hydro,FR0 65 hydro,FR0 68 hydro,FR0 70 hydro,FR0 71 hydro,FR0 72 hydro,FR0 74 hydro,FR0 8 hydro,FR0 86 hydro,FR0 87 hydro,FR0 89 hydro,GB5 19 hydro,GB5 4 hydro,GB5 45 hydro,GB5 50 hydro,GB5 61 hydro,GB5 87 hydro,GR0 1 hydro,GR0 11 hydro,GR0 3 hydro,GR0 4 hydro,GR0 6 hydro,GR0 9 hydro,HR0 0 hydro,HR0 1 hydro,HR0 3 hydro,HU0 6 hydro,IE4 2 hydro,IT0 1 hydro,IT0 12 hydro,IT0 14 hydro,IT0 18 hydro,IT0 2 hydro,IT0 22 hydro,IT0 25 hydro,IT0 31 hydro,IT0 33 hydro,IT0 34 hydro,IT0 36 hydro,IT0 38 hydro,IT0 45 hydro,IT0 46 hydro,IT0 47 hydro,IT0 50 hydro,IT0 52 hydro,IT0 54 hydro,IT0 56 hydro,IT0 58 hydro,IT0 59 hydro,IT0 6 hydro,IT0 61 hydro,IT0 64 hydro,IT0 69 hydro,IT0 7 hydro,IT0 70 hydro,IT0 71 hydro,IT0 81 hydro,IT0 84 hydro,IT0 91 hydro,IT0 92 hydro,IT1 0 hydro,IT1 1 hydro,IT1 2 hydro,LT6 0 hydro,LV6 1 hydro,MK0 0 hydro,MK0 1 hydro,NO3 0 hydro,NO3 1 hydro,NO3 10 hydro,NO3 11 hydro,NO3 12 hydro,NO3 13 hydro,NO3 14 hydro,NO3 15 hydro,NO3 16 hydro,NO3 17 hydro,NO3 18 hydro,NO3 19 hydro,NO3 2 hydro,NO3 20 hydro,NO3 21 hydro,NO3 22 hydro,NO3 23 hydro,NO3 24 hydro,NO3 25 hydro,NO3 26 hydro,NO3 27 hydro,NO3 29 hydro,NO3 3 hydro,NO3 30 hydro,NO3 32 hydro,NO3 33 hydro,NO3 34 hydro,NO3 36 hydro,NO3 37 hydro,NO3 38 hydro,NO3 4 hydro,NO3 5 hydro,NO3 6 hydro,NO3 7 hydro,NO3 8 hydro,NO3 9 hydro,PL0 14 hydro,PL0 16 hydro,PL0 24 hydro,PL0 26 hydro,PL0 28 hydro,PL0 3 hydro,PL0 33 hydro,PL0 42 hydro,PL0 8 hydro,PT0 0 hydro,PT0 10 hydro,PT0 11 hydro,PT0 12 hydro,PT0 13 hydro,PT0 2 hydro,PT0 4 hydro,PT0 5 hydro,PT0 6 hydro,PT0 8 hydro,RO0 0 hydro,RO0 1 hydro,RO0 10 hydro,RO0 12 hydro,RO0 13 hydro,RO0 14 hydro,RO0 15 hydro,RO0 16 hydro,RO0 2 hydro,RO0 3 hydro,RO0 5 hydro,RO0 6 hydro,RO0 7 hydro,RO0 8 hydro,RO0 9 hydro,SE3 0 hydro,SE3 1 hydro,SE3 10 hydro,SE3 11 hydro,SE3 12 hydro,SE3 13 hydro,SE3 14 hydro,SE3 15 hydro,SE3 16 hydro,SE3 18 hydro,SE3 19 hydro,SE3 2 hydro,SE3 24 hydro,SE3 27 hydro,SE3 28 hydro,SE3 29 hydro,SE3 31 hydro,SE3 35 hydro,SE3 36 hydro,SE3 4 hydro,SE3 5 hydro,SE3 8 hydro,SE3 9 hydro,SI0 0 hydro,SI0 1 hydro,SI0 3 hydro,SI0 4 hydro,SK0 6 hydro,SK0 8 hydro
1,22257.704761,13385.560250,523.214081,4970.533774,8807.437037,64691.061051,218.005867,13341.959076,39023.050239,56581.242789,39023.050239,4849.322511,2267.666037,391.687770,4149.570637,3561.200872,2617.687477,198.718327,92.212681,6491.816780,71446.745226,19017.476973,29114.665959,820.957348,22978.609443,2295.748585,9954.107848,20729.466243,17214.302797,2081.713276,12696.984988,15713.416845,73.299763,67895.225097,432.754637,472.095967,118.023992,6034.326807,629.461289,143

In [34]:
n52.storage_units

,efficiency_store,efficiency_dispatch,carrier,max_hours,bus,cyclic_state_of_charge,p_min_pu,p_nom,capital_cost,marginal_cost,build_year,control,type,p_nom_extendable,p_nom_min,p_nom_max,p_max_pu,p_set,q_set,sign,lifetime,state_of_charge_initial,state_of_charge_initial_per_period,state_of_charge_set,cyclic_state_of_charge_per_period,standing_loss,inflow,p_nom_opt
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AT0 0 PHS,0.866025,0.866025,PHS,0.022496,AT,True,-1.0,356160.000000,160627.143522,3.0,0,PQ,,False,0.0,inf,1.0,0.0,0.0,1.0,inf,0.0,False,NaN,True,0.0,0.0,0.0
AT0 0 hydro,0.000000,0.900000,hydro,2.698466,AT,True,0.0,85761.324841,0.000000,3.0,0,PQ,,False,0.0,inf,1.0,0.0,0.0,1.0,inf,0.0,False,NaN,True,0.0,0.0,0.0
AT0 10 hydro,0.000000,0.900000,hydro,2.698466,AT,True,0.0,51576.000000,0.000000,3.0,0,PQ,,False,0.0,inf,1.0,0.0,0.0,1.0,inf,0.0,False,NaN,True,0.0,0.0,0.0
AT0 13 hydro,0.000000,0.900000,hydro,2.698466,AT,True,0.0,2016.000000,0.000000,3.0,0,PQ,,False,0.0,inf,1.0,0.0,0.0,1.0,inf,0.0,False,NaN,True,0.0,0.0,0.0
AT0 14 hydro,0.000000,0.900000,hydro,2.698466,AT,True,0.0,19152.000000,0.000000,3.0,0,PQ,,False,0.0,inf,1.0,0.0,0.0,1.0,inf,0.0,False,NaN,True,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SI0 4 hydro,0.000000,0.900000,hydro,41.865910,SI,True,0.0,23352.000000,0.000000,3.0,0,PQ,,False,0.0,inf,1.0,0.0,0.0,1.0,inf,0.0,False,NaN,True,0.0,0.0,0.0
SK0 0 PHS,0.866025,0.866025,PHS,0.035714,SK,True,-1.0,156770.880000,160627.143522,3.0,0,PQ,,False,0.0,inf,1.0,0.0,0.0,1.0,inf,0.0,False,NaN,True,0.0,0.0,0.0
SK0 3 PHS,0.866025,0.866025,PHS,0.035714,SK,True,-1.0,14804.160000,160627.143522,3.0,0,PQ,,False,0.0,inf,1.0,0.0,0.0,1.0,inf,0.0,False,NaN,True,0.0,0.0,0.0


Links (no Lines in zonal network)

In [35]:
# links need to have extended capacities of 168 times more (hours in week)
n52.links = n.links
n52.links.p_nom = n52.links.p_nom*168

In [40]:
## export
path_save = 'D:\\Python\\PyPSA\\Luca\\zonal_nodal_networks\\2018\\reduced_hydro\\zonal_reduced_03_2018.nc'
n52.export_to_netcdf(path_save)

INFO:pypsa.io:Exported network zonal_reduced_03_2018.nc has storage_units, buses, generators, loads, links


<xarray.Dataset>
Dimensions:                               (buses_i: 45, generators_i: 2888, generators_t_p_max_pu_i: 2693, investment_periods: 0, links_i: 96, loads_i: 40, loads_t_p_set_i: 40, snapshots: 52, storage_units_i: 396, storage_units_t_inflow_i: 279)
Coordinates:
  * snapshots                             (snapshots) int64 0 1 2 3 ... 49 50 51
  * investment_periods                    (investment_periods) object 
  * storage_units_i                       (storage_units_i) object 'AT0 0 PHS...
  * storage_units_t_inflow_i              (storage_units_t_inflow_i) object '...
  * buses_i                               (buses_i) object 'AL' 'AT' ... 'ITCO'
  * generators_i                          (generators_i) object 'AT CCGT' ......
  * generators_t_p_max_pu_i               (generators_t_p_max_pu_i) object 'A...
  * loads_i                               (loads_i) object 'AT' 'BE' ... 'SK'
  * loads_t_p_set_i                       (loads_t_p_set_i) object 'AT' ... 'SK'
  * links_i                               (links_i) object 'AL-GR' ... 'SE3-SE4'
Data variables:
    snapshots_snapshot                    (snapshots) int64 1 2 3 4 ... 50 51 52
    snapshots_objective                   (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_stores                      (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_generators                  (snapshots) float64 1.0 1.0 ... 1.0
    investment_periods_objective          (investment_periods) object 
    investment_periods_years              (investment_periods) object 
    storage_units_efficiency_store        (storage_units_i) float64 0.866 ......
    storage_units_efficiency_dispatch     (storage_units_i) float64 0.866 ......
    storage_units_carrier                 (storage_units_i) object 'PHS' ... ...
    storage_units_max_hours               (storage_units_i) float64 0.0225 .....
    storage_units_bus                     (storage_units_i) object 'AT' ... 'SK'
    storage_units_cyclic_state_of_charge  (storage_units_i) bool True ... True
    storage_units_p_min_pu                (storage_units_i) float64 -1.0 ... 0.0
    storage_units_p_nom                   (storage_units_i) float64 3.562e+05...
    storage_units_capital_cost            (storage_units_i) float64 1.606e+05...
    storage_units_marginal_cost           (storage_units_i) float64 3.0 ... 3.0
    storage_units_t_inflow                (snapshots, storage_units_t_inflow_i) float64 ...
    buses_v_nom                           (buses_i) float64 380.0 ... 380.0
    buses_x                               (buses_i) float64 19.96 13.87 ... 9.0
    buses_y                               (buses_i) float64 41.38 47.56 ... 42.0
    buses_country                         (buses_i) object 'AL' 'AT' ... 'IT'
    buses_control                         (buses_i) object 'PQ' 'PQ' ... 'PQ'
    buses_generator                       (buses_i) object '' '' '' ... '' '' ''
    buses_zone                            (buses_i) object 'AL' 'AT' ... 'ITCO'
    generators_bus                        (generators_i) object 'AT' ... 'SK'
    generators_p_nom                      (generators_i) float64 4.169e+05 .....
    generators_carrier                    (generators_i) object 'CCGT' ... 's...
    generators_marginal_cost              (generators_i) float64 53.79 ... 0.0
    generators_build_year                 (generators_i) int64 0 0 0 0 ... 0 0 0
    generators_efficiency                 (generators_i) float64 0.5 ... 1.0
    generators_min_up_time                (generators_i) int64 0 0 0 0 ... 0 0 0
    generators_min_down_time              (generators_i) int64 0 0 0 0 ... 0 0 0
    generators_up_time_before             (generators_i) int64 1 1 1 1 ... 1 1 1
    generators_down_time_before           (generators_i) int64 0 0 0 0 ... 0 0 0
    generators_t_p_max_pu                 (snapshots, generators_t_p_max_pu_i) float64 ...
    loads_bus                             (loads_i) object 'AT' 'BE' ... 'SK'
    loads_t_p_set                

Later: full nodal network
Now check results

In [37]:
for c in n.iterate_components(list(n.components.keys())[2:]):
    print("Components '{}' has {} entries".format(c.name,len(c.df)))

Components 'Bus' has 45 entries
Components 'Carrier' has 15 entries
Components 'LineType' has 31 entries
Components 'TransformerType' has 14 entries
Components 'Link' has 96 entries
Components 'Load' has 973 entries
Components 'Generator' has 4639 entries
Components 'StorageUnit' has 396 entries


In [38]:
(n.buses)

,v_nom,x,y,country,control,generator,zone,type,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,sub_network
name,,,,,,,,,,,,,,
AL,380.0,19.962103,41.380653,AL,PQ,,AL,,AC,None,1.0,0.0,inf,
AT,380.0,13.872205,47.563063,AT,PQ,,AT,,AC,None,1.0,0.0,inf,
BA,380.0,17.936878,44.178532,BA,PQ,,BA,,AC,None,1.0,0.0,inf,
BE,380.0,4.650016,50.765858,BE,PQ,,BE,,AC,None,1.0,0.0,inf,
BG,380.0,25.307243,42.788639,BG,PQ,,BG,,AC,None,1.0,0.0,inf,
CH,380.0,7.997607,46.746162,CH,PQ,,CH,,AC,None,1.0,0.0,inf,
ITn,380.0,10.100551,45.298112,IT,PQ,,ITn,,AC,None,1.0,0.0,inf,
CZ,380.0,15.309982,49.775538,CZ,PQ,,CZ,,AC,None,1.0,0.0,inf,
DE,380.0,9.887726,50.758016,DE,PQ,,DE,,AC,None,1.0,0.0,inf,


In [39]:
(n52.loads_t.p_set)

,AT,BE,BG,CH,ITn,CZ,DE,DKw,DKe,EE,ES,FI,FR,GB,GR,HR,HU,IE,ITsic,ITs,ITcs,ITcn,ITsar,LT,LU,LV,MK,NL,NOm,NOs,NOn,PL,PT,RO,SE1,SE3,SE2,SE4,SI,SK
1,1.277460e+06,1.658259e+06,719629.38,1.237087e+06,2.657520e+06,1292294.0,9.750561e+06,396700.572413,302968.727587,169286.80,4579288.0,1.918860e+06,1.012203e+07,7348351.39,998847.03,339275.0,812676.18,577131.14,360226.765738,519674.698146,8.220662e+05,8.109282e+05,290098.770928,233338.0,139306.460828,143005.0,153959.0,2326043.0,539845.238352,2.299063e+06,295221.374611,3.018339e+06,995069.0,1035740.0,209417.316615,1.975939e+06,396686.310833,602951.347609,357489.536567,556335.18
2,1.400851e+06,1.804059e+06,777465.46,1.303459e+06,3.141572e+06,1397647.0,1.074173e+07,419194.600876,320147.899124,191148.20,5362617.0,2.057950e+06,1.125170e+07,7974986.35,1056235.21,350917.0,878505.16,612082.67,426043.175063,614623.563422,9.722645e+05,9.537663e+05,338874.005002,256595.0,154569.497399,157122.0,155940.0,2435448.0,557607.484444,2.374708e+06,304934.889402,3.382549e+06,1103806.0,1163656.0,224566.797667,2.118881e+06,425383.038718,646569.517124,393659.824085,603118.39
3,1.443220e+06,1.795370e+06,820095.32,1.323436e+06,3.220323e+06,1458526.0,1.089331e+07,423762.215622,323636.284378,201366.90,5226848.0,2.138736e+06,1.116139e+07,7802106.46,1075286.66,367573.0,908670.49,611329.78,436739.165123,630053.942103,9.966736e+05,9.749488e+05,345188.308452,262117.0,160127.051835,162993.0,163037.0,2445837.0,558492.727863,2.378478e+06,305418.996253,3.410239e+06,1072152.0,1217449.0,226409.542022,2.136268e+06,428873.635732,651875.120359,405388.836372,627998.13
4,1.421592e+06,1.745961e+06,844627.17,1.265844e+06,3.225561e+06,1439291.0,1.072204e+07,417176.481720,318606.618280,189226.40,5138450.0,2.071988e+06,1.069161e+07,7349547.52,1124366.97,362330.0,902015.72,592221.87,437525.782051,631188.740939,9.984687e+05,9.730730e+05,342926.403310,260761.0,158159.953147,160384.0,171492.0,2416206.0,545662.990408,2.323839e+06,298402.887824,3.365391e+06,1065821.0,1205605.0,215121.894517,2.029764e+06,407492.141025,619375.886845,399493.762396,621888.53
5,1.398765e+06,1.787668e+06,740929.33,1.285470e+06,3.215952e+06,1408212.0,1.087258e+07,424881.269894,324490.930106,178978.85,5270601.0,2.089116e+06,1.137796e+07,7540229.86,1053394.16,362892.0,883160.75,602700.58,436192.567380,629265.402648,9.954262e+05,9.754416e+05,346116.419056,251146.0,159567.927533,154759.0,152197.0,2432070.0,560798.846480,2.388299e+06,306680.127148,3.302571e+06,1060980.0,1165717.0,223866.349668,2.112272e+06,424056.223261,644552.797248,399828.960275,607903.26
6,1.439179e+06,1.849325e+06,750589.32,1.292836e+06,3.267375e+06,1474944.0,1.095489e+07,429000.060926,327636.539074,182464.80,5524032.0,2.156781e+06,1.277539e+07,7839340.00,1019729.80,381864.0,905435.39,610352.64,443184.139727,639351.669396,1.011382e+06,1.000097e+06,358826.604863,259302.0,159587.805584,161774.0,151119.0,2426195.0,566733.068037,2.413571e+06,309925.333220,3.415652e+06,1118280.0,1182914.0,237865.183955,2.244356e+06,450573.351925,684858.041030,397613.617825,623332.07
7,1.440773e+06,1.790061e+06,775693.26,1.282300e+06,3.281845e+06,1470870.0,1.087907e+07,414318.887590,316424.212410,202361.80,5245433.0,2.070679e+06,1.186365e+07,7559908.00,1052139.23,382634.0,906433.84,606953.26,445167.609075,642213.085962,1.015908e+06,9.973852e+05,354725.273753,248161.0,158943.694172,157136.0,156529.0,2363779.0,546726.073220,2.328366e+06,298984.248457,3.364078e+06,1036147.0,1187429.0,221139.448339,2.086542e+06,418890.822205,636701.541880,410363.873190,623735.26
8,1.471196e+06,1.797258e+06,806084.31,1.338252e+06,3.271705e+06,1515886.0,1.096445e+07,408746.805828,312168.694172,194642.20,5155073.0,2.245113e+06,1.221568e+07,7564742.00,1043159.42,395022.0,913823.69,598705.90,443715.601125,640118.372673,1.012594e+06,9.970323e+05,355871.143382,262578.0,159884.805436,164818.0,158401.0,2381028.0,556491.434089,2.369955e+06,304324.562781,3.410632e+06,1018817.0,1216882.0,236695.535081,2.233320e+06,448357.758181,681490.404694,419253.97604